In [ ]:
import tensorflow as tf
#tf.test.gpu_device_name()
from google.colab import drive

# 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install SpeechRecognition
!pip3 install wheel
!sudo apt-get install python python-all-dev python-pip build-essential swig git libpulse-dev libasound2-dev
!pip3 install pocketsphinx

!pip3 install -U sentence-transformers
!pip3 install jiwer
from jiwer import wer

In [ ]:
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
#from spectra_gen import *
#from to_rank import *
#from utils import *
#from datetime import datetime
#from mask import *
from itertools import combinations
import math
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import cv2
import time



In [ ]:
import speech_recognition as speech

import speech_recognition as speech

def transcribe_google(audiopath):

  recognizer = speech.Recognizer()
  harvard = speech.AudioFile(audiopath)  # 读取音频文件
  with harvard as source:
      audio = recognizer.record(source)
  # 使用google识别语音
  try:
      #print("google识别结果为:" + recognizer.recognize_google(audio))
      #换一下sphinx
      return recognizer.recognize_google(audio)
  except speech.UnknownValueError:
      print("无法识别音频")
  except speech.RequestError as e:
      print("识别错误; {0}".format(e))

In [ ]:
#暂时用一下，后面肯定要再改
import scipy.io.wavfile as wav
import copy
def store_new(flag,fs,x,path_store):
  data_final=copy.deepcopy(x)
  for i in range(len(flag)):
    if(flag[i]==0):
      data_final[512*i:512*(i+1)]=copy.deepcopy(512*[0])
  
  wav.write(path_store, fs, data_final)


  

In [ ]:
class boxt:
  def __init__(self, x, y):
    self.x, self.y = x , y

  def area(self):
      return self.y-self.x

class nodet:
  def __init__(self, heatMap, frags, x, y, inp, outp,fs, totScore, mask_value, depth):
    self.heatMap=heatMap
    self.frags=4 #frags
    self.x, self.y = x , y
    self.inp=inp
    self.outp=outp
    self.fs=fs
    self.totScore=totScore
    self.mask_value=mask_value
    self.fragSize_lb= 4 
    self.depth = depth

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

In [ ]:
!pip3 install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_bert(new_translation):
  #bert_embedding = BertEmbedding(),
  test=[new_translation]
  test_text_embed = model.encode(test)
  return test_text_embed

def bert_simi(ori_bert, new_bert):
  return cosine_similarity(ori_bert, new_bert)

In [ ]:
from sys import flags
import scipy.io.wavfile as wav
import pandas as pd
import os
import wave
import numpy as np
import copy
import random
#add a function to compare simialrity and get a true and flase and one score
def compare_tfidf(key, new_translation):

#比较tf-idf的时候需要的是新的一句翻译，然后对它进行分词，得到一个分词列表，然后通过key去查阅对应的字典及其概率，然后相加，得到正确或者错误以及extent
#需要：key,分词算法，translation
#return: boolean 和一个score
  if(new_translation==None):
    return False, 0.0
  dic_=np.load('/content/gdrive/MyDrive/play_deepcover/translation/commonvoice_tf_idf_normalized.npy',allow_pickle=True)
  tfidf_all=dic_.item()
  tfidf_ori=tfidf_all[key]
  words=new_translation.strip().split()
  score=0
  for i in range (len(words)):
    if(words[i] in tfidf_ori.keys()):
      score+=tfidf_ori[words[i]]
  if(score>0.5):
    #score=(score-0.5)/0.5
    return True, score
  else:
    #score=(0.5-score)/0.5
    return False, score 


def compare_bert(key, new_translation):
  if(new_translation==None):
    return False, 0.0
  dic_=np.load('/content/gdrive/MyDrive/play_deepcover/translation/commonvoice_bert.npy',allow_pickle=True)
  bert_all=dic_.item()
  ori_bert=bert_all[key]
  new_bert=get_bert(new_translation)
  score=bert_simi(ori_bert, new_bert)
  if(score>0.5):
    #score=(score-0.5)/0.5
    return True, score
  else:
    #score=(0.5-score)/0.5
    return False, score


In [ ]:
l=[1,2,3,5]
print(l[:-1])

[1, 2, 3]


In [ ]:
#这个待改动，不过这一段写的逻辑不错，配对的是论文的对应的伪代码
#整个逻辑很漂亮，因为要找的是那种自己没被masked的时候输出true，当其他的条件保持不变的时候，自己被masked之后输出false
#整个逻辑都是先找自己是masked掉然后输出是false，再在truthtable里面往combination的上一层（也就是少masked一个的）去找没被masked输出是true且整个改变只有这两个的
#666666666这个逻辑是真的牛皮，我还得再看一遍
#准备在truthtable上加上extent
#返回一个权重，原先是row的地方我加上【：-1】


def causal_search(i_feature, truthTable):
  all_true=[]
  #for i in range(0, len(truthTable)-1):我改了
  for i in range(0, len(truthTable)):
    all_true.append(True)
  for i in range(0, len(truthTable)):
    i_rows=truthTable[i]
    for row in i_rows:
      #改-2为-1，因为最后一个是相似度
      if row[i_feature] or row[-2:][0]: continue
      if i==0:
        return row[:-2], all_true, abs(1-row[-1:][0])
        #1/(1.+1)
        #我有道理觉得这里的all_true少了一个
      ## to search these rows with i-1 False
      minus_i_rows=truthTable[i-1]
      for row2 in minus_i_rows:
        if not row2[i_feature] or not row2[-2:][0]: continue
        xor_res=np.logical_xor(row[:-1], row2[:-1])
        if np.count_nonzero(xor_res)>2: continue
        c=len(row[:-1])-1-np.count_nonzero(row[:-1])
        return row[:-2], row2[:-2], abs(row2[-1:][0]-row[-1:][0])
  return None, None, 0

In [ ]:
#这个里面需要考虑的东西有点多，需要改变的东西有点多，那个length有点让我不知所措
#还是应该全都改成以帧计数的方式
def move(x1,x):
    k=(x1-x)%512
    if(k<=256):x1=x1-k
    else: x1=x1+(512-k)
    return x1


def compositional_causal_explain_tfidf(node,key):
  tmp = 1
  heatMap=np.zeros(node.heatMap.shape)
  frags=4 #node.frags
  x, y=node.x, node.y
  inp=node.inp
  outp=node.outp
  fs=node.fs
  mask_value=node.mask_value

  length=y-x
  print("length,depth")
  print(length,node.depth)

  final_boxes=None
  final_scores=None
  final_res_rows=None
  var_factor=-1
  max_factor=-1
  ave_factor=-1
  area_factor=10000#needs to be changed

  #if node.depth>2 or node.totScore<=10 or length<node.fragSize_lb or height<node.fragSize_lb: ## end point
  if length<node.fragSize_lb or node.depth>3 : 
  #if length<node.fragSize_lb :## end point
      #regionSize=heatMap[x1:x2,y1:y2,:].size #(x2-x1)*(y2-y1)*3
      regionSize=y-x
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  
  for s in range(0, 1): #step):
    boxes=[]
    #这里存在一些个问题需要改正
    x1 = np.random.randint(x+1,y)
    while(x1-x-1<1 or y-x1<2):
    #while(x+1==x1 or x1+1==y):
      x1=np.random.randint(x+1,y)
    x2 = np.random.randint(x+1,x1)
    x3 = np.random.randint(x1+1,y)
    box0=boxt(x, x2)
    box1=boxt(x2, x1)
    box2=boxt(x1, x3)
    box3=boxt(x3, y)
    boxes.append(box0)
    boxes.append(box1)
    boxes.append(box2)
    boxes.append(box3)
  
    # to build the truth table
    truthTable=[]
    n=4
    indices=np.arange(0, n)
    for r in range(1, n+1):
      comb_list = list(combinations(indices, r))
      rows_r=[]
      for comb in comb_list:
        early_stop = 0
        row=[]
        mutant=y*[1]
        for index in indices:
          if index in comb:
            row.append(False)
            print(mutant)
            mutant[boxes[index].x:boxes[index].y]=copy.deepcopy([0]*(boxes[index].y-boxes[index].x))
            early_stop = early_stop + 1
          else: row.append(True)
        
        #if early_stop<=2: continue

        #res=eobj.model.predict(sbfl_preprocess(eobj, np.array([mutant])))
        #get the new result of the mutant
        path_temp="/content/gdrive/MyDrive/play_deepcover/original/temp_DCcausal/"+key+"_temp.wav"
        print(mutant,inp)
        store_new(mutant,fs,inp,path_temp)
        y_mutant=transcribe_google(path_temp)
        print(y_mutant)
        print("===============rangwokk=============")
        still_right, score=compare_tfidf(key,y_mutant)
        if(still_right!=True):
          row.append(False)
          #加上一列分数，程度这里都是原始的similarity，好相减，需要修改函数，很简单
          row.append(score)
        else:
          row.append(True)
          row.append(score)
        #if not (y_mutant == outp):
         # row.append(False)
        #else:
         # row.append(True)
        rows_r.append(np.array(row))
      ##
      truthTable.append(np.array(rows_r))

    # to compute the scores,其实这里的都是一个小方块形式的，我的还是需要把他们都改成一列数据的形式
    # 感觉需要加上权重
    scores=np.zeros((frags,))
    res_rows=[]
    for i_feature in range(0, n):
      row, row2, weight=causal_search(i_feature, truthTable)
      print(weight)
      res_rows.append(row2)
      uIndex=np.unravel_index(i_feature, (frags,))
      res=0
      if row is not None:
        #res=(1./(len(row)-np.count_nonzero(row)+1))*weight
        #自己弄的
        res=(1./(len(row)-np.count_nonzero(row)+1))*1.0
      scores[uIndex]=res

    std=np.std(scores)
    ave=np.mean(scores)
    max_=np.max(scores)
    if std>var_factor:
      var_factor=std
      ave_factor=ave
      final_boxes=boxes
      final_scores=scores
      final_res_rows=res_rows

  if final_scores is not None:
    norm_sum=final_scores.sum()
    if norm_sum>0:
      final_scores=(final_scores/norm_sum)*node.totScore
    else:
        regionSize=len(heatMap[x:y]) 
        heatMap[x:y]=node.totScore/regionSize
        return heatMap

  if final_scores is None: 
      regionSize=len(heatMap[x:y])
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  else:
      for i in range(0, len(final_boxes)):
          box=final_boxes[i]
          uIndex=np.unravel_index(i, (4,))
          if final_scores[uIndex]<0.0001:
            heatMap[box.x:box.y]=0
            continue
          child_inp=inp.copy()
              
          child_node=nodet(heatMap, frags, box.x, box.y,child_inp, node.outp,node.fs, final_scores[uIndex], node.mask_value, node.depth+1)
          child_heatMap=compositional_causal_explain(child_node,key)
          heatMap[box.x:box.y]=child_heatMap[box.x:box.y]
                

  return heatMap

In [ ]:
#这个里面需要考虑的东西有点多，需要改变的东西有点多，那个length有点让我不知所措
#还是应该全都改成以帧计数的方式
def move(x1,x):
    k=(x1-x)%512
    if(k<=256):x1=x1-k
    else: x1=x1+(512-k)
    return x1


def compositional_causal_explain_bert(node,key):
  tmp = 1
  heatMap=np.zeros(node.heatMap.shape)
  frags=4 #node.frags
  x, y=node.x, node.y
  inp=node.inp
  outp=node.outp
  fs=node.fs
  mask_value=node.mask_value

  length=y-x
  print("length,depth")
  print(length,node.depth)

  final_boxes=None
  final_scores=None
  final_res_rows=None
  var_factor=-1
  max_factor=-1
  ave_factor=-1
  area_factor=10000#needs to be changed

  #if node.depth>2 or node.totScore<=10 or length<node.fragSize_lb or height<node.fragSize_lb: ## end point
  if length<node.fragSize_lb or node.depth>3 : 
  #if length<node.fragSize_lb :## end point
      #regionSize=heatMap[x1:x2,y1:y2,:].size #(x2-x1)*(y2-y1)*3
      regionSize=y-x
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  
  for s in range(0, 1): #step):
    boxes=[]
    #这里存在一些个问题需要改正
    x1 = np.random.randint(x+1,y)
    while(x1-x-1<1 or y-x1<2):
    #while(x+1==x1 or x1+1==y):
      x1=np.random.randint(x+1,y)
    x2 = np.random.randint(x+1,x1)
    x3 = np.random.randint(x1+1,y)
    box0=boxt(x, x2)
    box1=boxt(x2, x1)
    box2=boxt(x1, x3)
    box3=boxt(x3, y)
    boxes.append(box0)
    boxes.append(box1)
    boxes.append(box2)
    boxes.append(box3)
  
    # to build the truth table
    truthTable=[]
    n=4
    indices=np.arange(0, n)
    for r in range(1, n+1):
      comb_list = list(combinations(indices, r))
      rows_r=[]
      for comb in comb_list:
        early_stop = 0
        row=[]
        mutant=y*[1]
        for index in indices:
          if index in comb:
            row.append(False)
            print(mutant)
            mutant[boxes[index].x:boxes[index].y]=copy.deepcopy([0]*(boxes[index].y-boxes[index].x))
            early_stop = early_stop + 1
          else: row.append(True)
        
        #if early_stop<=2: continue

        #res=eobj.model.predict(sbfl_preprocess(eobj, np.array([mutant])))
        #get the new result of the mutant
        path_temp="/content/gdrive/MyDrive/play_deepcover/DC_commonvoice_temp_tfidf/"+key+"_temp.wav"
        print(mutant,inp)
        store_new(mutant,fs,inp,path_temp)
        y_mutant=transcribe_google(path_temp)
        print(y_mutant)
        print("===============rangwokk=============")
        still_right, score=compare_bert(key,y_mutant)
        if(still_right!=True):
          row.append(False)
          #加上一列分数，程度这里都是原始的similarity，好相减，需要修改函数，很简单
          row.append(score)
        else:
          row.append(True)
          row.append(score)
        #if not (y_mutant == outp):
         # row.append(False)
        #else:
         # row.append(True)
        rows_r.append(np.array(row))
      ##
      truthTable.append(np.array(rows_r))

    # to compute the scores,其实这里的都是一个小方块形式的，我的还是需要把他们都改成一列数据的形式
    # 感觉需要加上权重
    scores=np.zeros((frags,))
    res_rows=[]
    for i_feature in range(0, n):
      row, row2, weight=causal_search(i_feature, truthTable)
      print(weight)
      res_rows.append(row2)
      uIndex=np.unravel_index(i_feature, (frags,))
      res=0
      if row is not None:
        #res=(1./(len(row)-np.count_nonzero(row)+1))*weight
        #自己弄的
        res=(1./(len(row)-np.count_nonzero(row)+1))*1.0
      scores[uIndex]=res

    std=np.std(scores)
    ave=np.mean(scores)
    max_=np.max(scores)
    if std>var_factor:
      var_factor=std
      ave_factor=ave
      final_boxes=boxes
      final_scores=scores
      final_res_rows=res_rows

  if final_scores is not None:
    norm_sum=final_scores.sum()
    if norm_sum>0:
      final_scores=(final_scores/norm_sum)*node.totScore
    else:
        regionSize=len(heatMap[x:y]) 
        heatMap[x:y]=node.totScore/regionSize
        return heatMap

  if final_scores is None: 
      regionSize=len(heatMap[x:y])
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  else:
      for i in range(0, len(final_boxes)):
          box=final_boxes[i]
          uIndex=np.unravel_index(i, (4,))
          if final_scores[uIndex]<0.0001:
            heatMap[box.x:box.y]=0
            continue
          child_inp=inp.copy()
              
          child_node=nodet(heatMap, frags, box.x, box.y,child_inp, node.outp,node.fs, final_scores[uIndex], node.mask_value, node.depth+1)
          child_heatMap=compositional_causal_explain(child_node,key)
          heatMap[box.x:box.y]=child_heatMap[box.x:box.y]
                

  return heatMap

In [ ]:
#这个里面需要考虑的东西有点多，需要改变的东西有点多，那个length有点让我不知所措
#还是应该全都改成以帧计数的方式
def move(x1,x):
    k=(x1-x)%512
    if(k<=256):x1=x1-k
    else: x1=x1+(512-k)
    return x1


def compositional_causal_explain_wer(node,key):
  tmp = 1
  heatMap=np.zeros(node.heatMap.shape)
  frags=4 #node.frags
  x, y=node.x, node.y
  inp=node.inp
  outp=node.outp
  fs=node.fs
  mask_value=node.mask_value

  length=y-x
  print("length,depth")
  print(length,node.depth)

  final_boxes=None
  final_scores=None
  final_res_rows=None
  var_factor=-1
  max_factor=-1
  ave_factor=-1
  area_factor=10000#needs to be changed

  #if node.depth>2 or node.totScore<=10 or length<node.fragSize_lb or height<node.fragSize_lb: ## end point
  if length<node.fragSize_lb or node.depth>3 : 
  #if length<node.fragSize_lb :## end point
      #regionSize=heatMap[x1:x2,y1:y2,:].size #(x2-x1)*(y2-y1)*3
      regionSize=y-x
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  
  for s in range(0, 1): #step):
    boxes=[]
    #这里存在一些个问题需要改正
    x1 = np.random.randint(x+1,y)
    while(x1-x-1<1 or y-x1<2):
    #while(x+1==x1 or x1+1==y):
      x1=np.random.randint(x+1,y)
    x2 = np.random.randint(x+1,x1)
    x3 = np.random.randint(x1+1,y)
    box0=boxt(x, x2)
    box1=boxt(x2, x1)
    box2=boxt(x1, x3)
    box3=boxt(x3, y)
    boxes.append(box0)
    boxes.append(box1)
    boxes.append(box2)
    boxes.append(box3)
  
    # to build the truth table
    truthTable=[]
    n=4
    indices=np.arange(0, n)
    for r in range(1, n+1):
      comb_list = list(combinations(indices, r))
      rows_r=[]
      for comb in comb_list:
        early_stop = 0
        row=[]
        mutant=y*[1]
        for index in indices:
          if index in comb:
            row.append(False)
            print(mutant)
            mutant[boxes[index].x:boxes[index].y]=copy.deepcopy([0]*(boxes[index].y-boxes[index].x))
            early_stop = early_stop + 1
          else: row.append(True)
        
        #if early_stop<=2: continue

        #res=eobj.model.predict(sbfl_preprocess(eobj, np.array([mutant])))
        #get the new result of the mutant
        path_temp="/content/gdrive/MyDrive/play_deepcover/original/temp_DCcausal_im/"+key+"_temp.wav"
        print(mutant,inp)
        store_new(mutant,fs,inp,path_temp)
        y_mutant=transcribe_google(path_temp)
        print(y_mutant)
        print("===============rangwokk=============")

        #still_right, score=compare_tfidf(key,y_mutant)
        if(y_mutant!=None):
          wer_=wer(transcription_list[key],y_mutant),
          if wer_==0.0:
            still_right=True
          else:
            still_right=False
        else:
          still_right=False

        score=1
        if(still_right!=True):
          row.append(False)
          #加上一列分数，程度这里都是原始的similarity，好相减，需要修改函数，很简单
          row.append(score)
        else:
          row.append(True)
          row.append(score)
        #if not (y_mutant == outp):
         # row.append(False)
        #else:
         # row.append(True)
        rows_r.append(np.array(row))
      ##
      truthTable.append(np.array(rows_r))

    # to compute the scores,其实这里的都是一个小方块形式的，我的还是需要把他们都改成一列数据的形式
    # 感觉需要加上权重
    scores=np.zeros((frags,))
    res_rows=[]
    for i_feature in range(0, n):
      row, row2, weight=causal_search(i_feature, truthTable)
      print(weight)
      res_rows.append(row2)
      uIndex=np.unravel_index(i_feature, (frags,))
      res=0
      if row is not None:
        #res=(1./(len(row)-np.count_nonzero(row)+1))*weight
        #自己弄的
        res=(1./(len(row)-np.count_nonzero(row)+1))*1.0
      scores[uIndex]=res

    std=np.std(scores)
    ave=np.mean(scores)
    max_=np.max(scores)
    if std>var_factor:
      var_factor=std
      ave_factor=ave
      final_boxes=boxes
      final_scores=scores
      final_res_rows=res_rows

  if final_scores is not None:
    norm_sum=final_scores.sum()
    if norm_sum>0:
      final_scores=(final_scores/norm_sum)*node.totScore
    else:
        regionSize=len(heatMap[x:y]) 
        heatMap[x:y]=node.totScore/regionSize
        return heatMap

  if final_scores is None: 
      regionSize=len(heatMap[x:y])
      heatMap[x:y]=node.totScore/regionSize
      return heatMap
  else:
      for i in range(0, len(final_boxes)):
          box=final_boxes[i]
          uIndex=np.unravel_index(i, (4,))
          if final_scores[uIndex]<0.0001:
            heatMap[box.x:box.y]=0
            continue
          child_inp=inp.copy()
              
          child_node=nodet(heatMap, frags, box.x, box.y,child_inp, node.outp,node.fs, final_scores[uIndex], node.mask_value, node.depth+1)
          child_heatMap=compositional_causal_explain(child_node,key)
          heatMap[box.x:box.y]=child_heatMap[box.x:box.y]
                

  return heatMap

In [ ]:
path_x='sample.wav'

y=transcription_google(path_x)

path_store="path_to_mutants"

fs, x = wav.read(path_x)
#给人家把那些index啥的全删了
x=np.array(x)
heatMap=np.zeros((int((len(x)-1536)//512),)) # initialise an all-zero heatmap
frags=4 #3 ## 3x3 is the limit an exhuastive search can handle
x1, y1=0, int((len(x)-1536)//512)-1
print(y1)
totScore = 10000.
#heatMap=np.ones(x.shape) * (totScore / heatMap.size)
node=nodet(heatMap, frags, x1, y1,x,y,fs, totScore, mask_value=0, depth=0)
# to call the recursive 'explain' method
start = time.time()
res_heatMap=compositional_causal_explain_tfidf(node,key)
print(res_heatMap)

124
length,depth
124 0
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] [ 0  0  0 ...  3 25 23]
learn to recognize Omens and follow them the Old King instead
===============rangwokk=============
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,